In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
%matplotlib inline
import dateutil.parser
import math
from datetime import date
from datetime import datetime, timedelta
import altair as alt
alt.data_transformers.disable_max_rows()
alt.data_transformers.enable('json')
import seaborn as sns
import re

In [2]:
today = date.today()
print(today)

2020-10-10


In [3]:
# source: https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide
ecdc = pd.read_csv("https://opendata.ecdc.europa.eu/covid19/casedistribution/csv", encoding = "utf-8")
ecdc.head()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
0,09/10/2020,9,10,2020,77,2,Afghanistan,AF,AFG,38041757.0,Asia,1.332746
1,08/10/2020,8,10,2020,68,1,Afghanistan,AF,AFG,38041757.0,Asia,1.172396
2,07/10/2020,7,10,2020,62,2,Afghanistan,AF,AFG,38041757.0,Asia,1.059362
3,06/10/2020,6,10,2020,145,5,Afghanistan,AF,AFG,38041757.0,Asia,1.083020
4,05/10/2020,5,10,2020,44,0,Afghanistan,AF,AFG,38041757.0,Asia,0.780721


In [4]:
# Comment/uncomment the following to show result for the World or a specific continent
ecdc = ecdc[ecdc['continentExp'] == 'Europe']

In [5]:
#excluding NANs
countries_list = ecdc[ecdc['countryterritoryCode'].notnull()]['countryterritoryCode'].unique()

In [6]:
ecdc['parsed_date'] = [re.sub(r'(\d\d)\/(\d\d)\/(\d\d\d\d)','\g<3>-\g<2>-\g<1>', date) for date in ecdc['dateRep']]
ecdc.head()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000,parsed_date
274,09/10/2020,9,10,2020,169,4,Albania,AL,ALB,2862427.0,Europe,69.102199,2020-10-09
275,08/10/2020,8,10,2020,162,4,Albania,AL,ALB,2862427.0,Europe,67.879460,2020-10-08
276,07/10/2020,7,10,2020,158,3,Albania,AL,ALB,2862427.0,Europe,66.447109,2020-10-07
277,06/10/2020,6,10,2020,144,4,Albania,AL,ALB,2862427.0,Europe,65.503854,2020-10-06
278,05/10/2020,5,10,2020,149,4,Albania,AL,ALB,2862427.0,Europe,65.713466,2020-10-05


In [7]:
ecdc['parsed_date'] = [datetime.strptime(date, '%Y-%m-%d') for date in ecdc['parsed_date']]

# Find all fridays

In [8]:
#https://stackoverflow.com/questions/2003870/how-can-i-select-all-of-the-sundays-for-a-year-using-python
from datetime import date, timedelta

def allfridays(year):
    d = date(year, 1, 1)                    # January 1st
    d += timedelta(days = 4 - d.weekday())  # First friday
    while d.year == year:
        yield d
        d += timedelta(days = 7)

In [9]:
for d in allfridays(2020):
    print(d)

2020-01-03
2020-01-10
2020-01-17
2020-01-24
2020-01-31
2020-02-07
2020-02-14
2020-02-21
2020-02-28
2020-03-06
2020-03-13
2020-03-20
2020-03-27
2020-04-03
2020-04-10
2020-04-17
2020-04-24
2020-05-01
2020-05-08
2020-05-15
2020-05-22
2020-05-29
2020-06-05
2020-06-12
2020-06-19
2020-06-26
2020-07-03
2020-07-10
2020-07-17
2020-07-24
2020-07-31
2020-08-07
2020-08-14
2020-08-21
2020-08-28
2020-09-04
2020-09-11
2020-09-18
2020-09-25
2020-10-02
2020-10-09
2020-10-16
2020-10-23
2020-10-30
2020-11-06
2020-11-13
2020-11-20
2020-11-27
2020-12-04
2020-12-11
2020-12-18
2020-12-25


In [10]:
# create class variables
moredouble = '01_moredouble'
more = '02_more'
samesame ='03_samesame'
less = '04_less'
lesshalf = '05_lesshalf'
zerozero ='06_zerozero'

In [11]:
threshold_samesame = 7

In [12]:
friday_data_list = []

for country in countries_list:
    #filter dataframe for one specific country at a time
    filtered_aggregates = ecdc[ecdc['countryterritoryCode']==country]
    for this_friday in allfridays(2020):
        country_dict = {}
        #filter country-specific-dataframe to only contain the last seven days ("this week")
        country_dict['country_ISO'] = country
        pd_this_friday = pd.to_datetime(this_friday)
        seven_days_ago = pd.to_datetime(this_friday-timedelta(7))
        country_dict['which_friday'] = this_friday
        country_dict['seven_days_ago'] = seven_days_ago
        filtered_aggregates_this_week = filtered_aggregates[(filtered_aggregates['parsed_date']<=pd_this_friday) & (filtered_aggregates['parsed_date']>seven_days_ago)]
        cases_this_week = filtered_aggregates_this_week['cases'].sum()
        country_dict['cases_this_week']= cases_this_week
        #filter country-specific-dataframe to only contain the previous seven days ("last week")    
        fourteen_day_ago = pd.to_datetime(this_friday-timedelta(14))
        country_dict['fourteen_days_ago'] = fourteen_day_ago
        filtered_aggregates_last_week = filtered_aggregates[(filtered_aggregates['parsed_date']<=seven_days_ago) &(filtered_aggregates['parsed_date']>fourteen_day_ago)]
        cases_last_week = filtered_aggregates_last_week['cases'].sum()
        country_dict['cases_last_week']= cases_last_week
        # classify country
        if (cases_last_week>0)&(cases_this_week > (cases_last_week * 2)):
            country_dict['trend_class'] = moredouble
        elif cases_this_week >= (cases_last_week+threshold_samesame):
            country_dict['trend_class'] = more
        elif (cases_last_week == 0)& (cases_this_week !=0):
            country_dict['trend_class'] = more
        elif (cases_last_week==0)&(cases_this_week==0):
            country_dict['trend_class'] = zerozero
        elif (cases_this_week == cases_last_week):
            country_dict['trend_class'] = samesame
        elif (cases_this_week == 0)&(cases_last_week != 0):
            country_dict['trend_class'] = less
        elif cases_last_week+threshold_samesame > cases_this_week > cases_last_week -threshold_samesame:
            country_dict['trend_class'] = samesame
        elif cases_this_week < cases_last_week/2:
            country_dict['trend_class'] = lesshalf
        elif cases_this_week <= cases_last_week-threshold_samesame:
            country_dict['trend_class'] = less
        else:
            country_dict['trend_class'] = 'youMISSEDsth'
        # add it all to the list of country dicts
        friday_data_list.append(country_dict)


In [13]:
all_week_trend = pd.DataFrame(friday_data_list)
all_week_trend.head()

,country_ISO,which_friday,seven_days_ago,cases_this_week,fourteen_days_ago,cases_last_week,trend_class
0,ALB,2020-01-03,2019-12-27,0,2019-12-20,0,06_zerozero
1,ALB,2020-01-10,2020-01-03,0,2019-12-27,0,06_zerozero
2,ALB,2020-01-17,2020-01-10,0,2020-01-03,0,06_zerozero
3,ALB,2020-01-24,2020-01-17,0,2020-01-10,0,06_zerozero
4,ALB,2020-01-31,2020-01-24,0,2020-01-17,0,06_zerozero


In [14]:
#filter to contain only data points up until the latest available friday
all_week_trend_data = all_week_trend[all_week_trend['which_friday'] <= today]
all_week_trend_data.head()

,country_ISO,which_friday,seven_days_ago,cases_this_week,fourteen_days_ago,cases_last_week,trend_class
0,ALB,2020-01-03,2019-12-27,0,2019-12-20,0,06_zerozero
1,ALB,2020-01-10,2020-01-03,0,2019-12-27,0,06_zerozero
2,ALB,2020-01-17,2020-01-10,0,2020-01-03,0,06_zerozero
3,ALB,2020-01-24,2020-01-17,0,2020-01-10,0,06_zerozero
4,ALB,2020-01-31,2020-01-24,0,2020-01-17,0,06_zerozero


In [15]:
#refactor data for plotting
value_counts = pd.DataFrame()

for friday in all_week_trend_data['which_friday']:
    weekly_trend = all_week_trend_data[all_week_trend_data['which_friday']==friday]
    weekly_trend_df = weekly_trend['trend_class'].value_counts().rename_axis('trend_class').reset_index(name='#_countries_in_class')
    weekly_trend_df['time'] = friday
    value_counts = pd.concat([value_counts, weekly_trend_df])

In [16]:
value_counts.head()

,trend_class,#_countries_in_class,time
0,06_zerozero,54,2020-01-03
0,06_zerozero,54,2020-01-10
0,06_zerozero,54,2020-01-17
0,06_zerozero,54,2020-01-24
0,06_zerozero,50,2020-01-31


In [17]:
value_counts['calendar_week'] = [day.isocalendar()[1] for day in value_counts['time']]

In [18]:
bars = alt.Chart(value_counts).mark_bar().encode(
    x=alt.X('calendar_week:N'),
    y=alt.Y('sum(#_countries_in_class):Q', stack='zero', axis = None),
    color=alt.Color('trend_class:N',scale=alt.Scale(domain=[moredouble, more,samesame,less, lesshalf,zerozero],range=["#BE232D","#D44820","#EE8C0A","#F0AA00","#F0C80F","#00A5FF"]))
)

bars.properties(width=400, height=400)

alt.Chart(...)

# Now do the same for deaths

In [19]:
friday_deaths_list = []

for country in countries_list:
    #filter dataframe for one specific country at a time
    filtered_aggregates = ecdc[ecdc['countryterritoryCode']==country]
    for this_friday in allfridays(2020):
        country_dict = {}
        #filter country-specific-dataframe to only contain the last seven days ("this week")
        country_dict['country_ISO'] = country
        pd_this_friday = pd.to_datetime(this_friday)
        seven_days_ago = pd.to_datetime(this_friday-timedelta(7))
        country_dict['which_friday'] = this_friday
        country_dict['seven_days_ago'] = seven_days_ago
        filtered_aggregates_this_week = filtered_aggregates[(filtered_aggregates['parsed_date']<=pd_this_friday) & (filtered_aggregates['parsed_date']>seven_days_ago)]
        deaths_this_week = filtered_aggregates_this_week['deaths'].sum()
        country_dict['deaths_this_week']= deaths_this_week
        #filter country-specific-dataframe to only contain the previous seven days ("last week")    
        fourteen_day_ago = pd.to_datetime(this_friday-timedelta(14))
        country_dict['fourteen_days_ago'] = fourteen_day_ago
        filtered_aggregates_last_week = filtered_aggregates[(filtered_aggregates['parsed_date']<=seven_days_ago) &(filtered_aggregates['parsed_date']>fourteen_day_ago)]
        deaths_last_week = filtered_aggregates_last_week['deaths'].sum()
        country_dict['deaths_last_week']= deaths_last_week
        # classify country
        if (deaths_last_week>0)&(deaths_this_week > (deaths_last_week * 2)):
            country_dict['trend_class'] = moredouble
        elif deaths_this_week >= (deaths_last_week+threshold_samesame):
            country_dict['trend_class'] = more
        elif (deaths_last_week == 0)& (deaths_this_week !=0):
            country_dict['trend_class'] = more
        elif (deaths_last_week==0)&(deaths_this_week==0):
            country_dict['trend_class'] = zerozero
        elif (deaths_this_week == deaths_last_week):
            country_dict['trend_class'] = samesame
        elif (deaths_this_week == 0)&(deaths_last_week != 0):
            country_dict['trend_class'] = less
        elif deaths_last_week+threshold_samesame > deaths_this_week > deaths_last_week -threshold_samesame:
            country_dict['trend_class'] = samesame
        elif deaths_this_week < deaths_last_week/2:
            country_dict['trend_class'] = lesshalf
        elif deaths_this_week <= deaths_last_week-threshold_samesame:
            country_dict['trend_class'] = less
        else:
            country_dict['trend_class'] = 'youMISSEDsth'
        # add it all to the list of country dicts
        friday_deaths_list.append(country_dict)

In [20]:
all_week_deaths_trend = pd.DataFrame(friday_deaths_list)
all_week_deaths_trend.head()

,country_ISO,which_friday,seven_days_ago,deaths_this_week,fourteen_days_ago,deaths_last_week,trend_class
0,ALB,2020-01-03,2019-12-27,0,2019-12-20,0,06_zerozero
1,ALB,2020-01-10,2020-01-03,0,2019-12-27,0,06_zerozero
2,ALB,2020-01-17,2020-01-10,0,2020-01-03,0,06_zerozero
3,ALB,2020-01-24,2020-01-17,0,2020-01-10,0,06_zerozero
4,ALB,2020-01-31,2020-01-24,0,2020-01-17,0,06_zerozero


In [21]:
#filter to contain only data points up until the latest available friday
all_week_trend_deaths = all_week_deaths_trend[all_week_deaths_trend['which_friday'] <= today]
all_week_trend_deaths.head()

,country_ISO,which_friday,seven_days_ago,deaths_this_week,fourteen_days_ago,deaths_last_week,trend_class
0,ALB,2020-01-03,2019-12-27,0,2019-12-20,0,06_zerozero
1,ALB,2020-01-10,2020-01-03,0,2019-12-27,0,06_zerozero
2,ALB,2020-01-17,2020-01-10,0,2020-01-03,0,06_zerozero
3,ALB,2020-01-24,2020-01-17,0,2020-01-10,0,06_zerozero
4,ALB,2020-01-31,2020-01-24,0,2020-01-17,0,06_zerozero


In [22]:
#refactor data for plotting
value_counts = pd.DataFrame()

for friday in all_week_trend_deaths['which_friday']:
    weekly_trend = all_week_trend_deaths[all_week_trend_deaths['which_friday']==friday]
    weekly_trend_df = weekly_trend['trend_class'].value_counts().rename_axis('trend_class').reset_index(name='#_countries_in_class')
    weekly_trend_df['time'] = friday
    value_counts = pd.concat([value_counts, weekly_trend_df])

In [23]:
value_counts.head()

,trend_class,#_countries_in_class,time
0,06_zerozero,54,2020-01-03
0,06_zerozero,54,2020-01-10
0,06_zerozero,54,2020-01-17
0,06_zerozero,54,2020-01-24
0,06_zerozero,54,2020-01-31


In [24]:
value_counts['calendar_week'] = [day.isocalendar()[1] for day in value_counts['time']]

In [25]:
bars = alt.Chart(value_counts).mark_bar().encode(
    x=alt.X('calendar_week:N'),
    y=alt.Y('sum(#_countries_in_class):Q', stack='zero', axis = None),
    color=alt.Color('trend_class:N',scale=alt.Scale(domain=[moredouble, more,samesame,less, lesshalf,zerozero],range=["#BE232D","#D44820","#EE8C0A","#F0AA00","#F0C80F","#00A5FF"]))
)

bars.properties(width=400, height=400)

alt.Chart(...)

# Now do the comparison with the average, not with previous week

In [26]:
friday_deaths_list = []

total_weeks = 0
for d in allfridays(2020):
    if(d <= today):
        total_weeks = total_weeks + 1
        
for country in countries_list:
    #filter dataframe for one specific country at a time
    filtered_aggregates = ecdc[ecdc['countryterritoryCode']==country]
    
    # Get date of first death to count the weeks correctly
    filtered_aggregates = filtered_aggregates.sort_values(by = ['parsed_date'], ascending=True)
    first_death_date = date.today()
    for i in filtered_aggregates.index.values:
        if(filtered_aggregates.loc[i, 'deaths'] > 0):
            first_death_date = filtered_aggregates.loc[i, 'parsed_date']
            break
    
    # How many weeks since first death
    total_weeks = 0
    for d in allfridays(2020):
        if(pd.to_datetime(d) > first_death_date and d <= today ):
            total_weeks = total_weeks + 1
    
    if(total_weeks == 0):
        total_weeks = 1 # To avoid division by zero
    deaths_average = filtered_aggregates['deaths'].sum() / total_weeks
    for this_friday in allfridays(2020):
        country_dict = {}
        #filter country-specific-dataframe to only contain the last seven days ("this week")
        country_dict['country_ISO'] = country
        country_dict['average'] = deaths_average
        country_dict['weeks'] = total_weeks
        pd_this_friday = pd.to_datetime(this_friday)
        seven_days_ago = pd.to_datetime(this_friday-timedelta(7))
        country_dict['which_friday'] = this_friday
        country_dict['seven_days_ago'] = seven_days_ago
        filtered_aggregates_this_week = filtered_aggregates[(filtered_aggregates['parsed_date']<=pd_this_friday) & (filtered_aggregates['parsed_date']>seven_days_ago)]
        deaths_this_week = filtered_aggregates_this_week['deaths'].sum()
        country_dict['deaths_this_week']= deaths_this_week
        #filter country-specific-dataframe to only contain the previous seven days ("last week")    
        #fourteen_day_ago = pd.to_datetime(this_friday-timedelta(14))
        #country_dict['fourteen_days_ago'] = fourteen_day_ago
        #filtered_aggregates_last_week = filtered_aggregates[(filtered_aggregates['parsed_date']<=seven_days_ago) &(filtered_aggregates['parsed_date']>fourteen_day_ago)]
        #deaths_average = filtered_aggregates['deaths'].mean()
        #country_dict['deaths_average']= deaths_average
        # classify country
        if (deaths_this_week == 0):
            country_dict['trend_class'] = zerozero
        elif (deaths_this_week > (deaths_average * 2)):
            country_dict['trend_class'] = moredouble
        elif deaths_this_week >= (deaths_average+threshold_samesame):
            country_dict['trend_class'] = more
        elif (deaths_average == 0)& (deaths_this_week !=0):
            country_dict['trend_class'] = more
        elif (deaths_this_week == deaths_average):
            country_dict['trend_class'] = samesame
        elif deaths_average+threshold_samesame > deaths_this_week > deaths_average -threshold_samesame:
            country_dict['trend_class'] = samesame
        elif deaths_this_week < deaths_average/2:
            country_dict['trend_class'] = lesshalf
        elif deaths_this_week <= deaths_average-threshold_samesame:
            country_dict['trend_class'] = less
        else:
            country_dict['trend_class'] = 'youMISSEDsth'
        # add it all to the list of country dicts
        friday_deaths_list.append(country_dict)

In [27]:
all_week_deaths_trend = pd.DataFrame(friday_deaths_list)
#all_week_deaths_trend.head()

#filter to contain only data points up until the latest available friday
all_week_trend_deaths = all_week_deaths_trend[all_week_deaths_trend['which_friday'] <= today]
#all_week_trend_deaths.head()

#refactor data for plotting
value_counts = pd.DataFrame()

for friday in all_week_trend_deaths['which_friday']:
    weekly_trend = all_week_trend_deaths[all_week_trend_deaths['which_friday']==friday]
    weekly_trend_df = weekly_trend['trend_class'].value_counts().rename_axis('trend_class').reset_index(name='#_countries_in_class')
    weekly_trend_df['time'] = friday
    value_counts = pd.concat([value_counts, weekly_trend_df])

this_week_countries_in_class = value_counts[value_counts['time'] == friday]

this_week_countries_in_class.head(10)    

,trend_class,#_countries_in_class,time
0,01_moredouble,14,2020-10-09
1,05_lesshalf,12,2020-10-09
2,06_zerozero,10,2020-10-09
3,03_samesame,7,2020-10-09
4,02_more,7,2020-10-09
5,04_less,4,2020-10-09
0,01_moredouble,14,2020-10-09
1,05_lesshalf,12,2020-10-09
2,06_zerozero,10,2020-10-09
3,03_samesame,7,2020-10-09


In [28]:
value_counts['calendar_week'] = [day.isocalendar()[1] for day in value_counts['time']]
bars = alt.Chart(value_counts).mark_bar().encode(
    x=alt.X('calendar_week:N'),
    y=alt.Y('sum(#_countries_in_class):Q', stack='zero', axis = None),
    color=alt.Color('trend_class:N',scale=alt.Scale(domain=[moredouble, more,samesame,less, lesshalf,zerozero],range=["#BE232D","#D44820","#EE8C0A","#F0AA00","#F0C80F","#00A5FF"]))
)

bars.properties(width=400, height=400)

alt.Chart(...)

In [29]:
# Countries with more than double the average
countries_in_class = all_week_trend_deaths[all_week_trend_deaths['trend_class'] == moredouble]
countries_in_class = countries_in_class[countries_in_class['which_friday'] == friday]
countries_in_class.head(20)

,country_ISO,average,weeks,which_friday,seven_days_ago,deaths_this_week,trend_class
508,HRV,10.689655,29,2020-10-09,2020-10-02,26,01_moredouble
560,CYP,0.827586,29,2020-10-09,2020-10-02,2,01_moredouble
612,CZE,29.965517,29,2020-10-09,2020-10-02,191,01_moredouble
924,GEO,2.444444,27,2020-10-09,2020-10-02,23,01_moredouble
1080,GRC,13.870968,31,2020-10-09,2020-10-02,37,01_moredouble
1236,HUN,30.433333,30,2020-10-09,2020-10-02,115,01_moredouble
1600,LVA,1.481481,27,2020-10-09,2020-10-02,3,01_moredouble
1704,LTU,3.517241,29,2020-10-09,2020-10-02,10,01_moredouble
1808,MLT,1.576923,26,2020-10-09,2020-10-02,6,01_moredouble
1964,MNE,6.821429,28,2020-10-09,2020-10-02,21,01_moredouble


In [30]:
# Worst top 20 trending countries   
all_week_trend_deaths['trend'] = all_week_trend_deaths['deaths_this_week'] / all_week_trend_deaths['average']
this_week_trend = all_week_trend_deaths[all_week_trend_deaths['which_friday'] == friday]
this_week_trend = this_week_trend.sort_values(by = ['trend'], ascending=False)
this_week_trend.head(20)

,country_ISO,average,weeks,which_friday,seven_days_ago,deaths_this_week,trend_class,trend
924,GEO,2.444444,27,2020-10-09,2020-10-02,23,01_moredouble,9.409091
612,CZE,29.965517,29,2020-10-09,2020-10-02,191,01_moredouble,6.373993
2484,SVK,2.111111,27,2020-10-09,2020-10-02,9,01_moredouble,4.263158
1808,MLT,1.576923,26,2020-10-09,2020-10-02,6,01_moredouble,3.804878
1236,HUN,30.433333,30,2020-10-09,2020-10-02,115,01_moredouble,3.778751
2172,POL,95.566667,30,2020-10-09,2020-10-02,324,01_moredouble,3.390303
2744,UKR,156.333333,30,2020-10-09,2020-10-02,497,01_moredouble,3.179104
1964,MNE,6.821429,28,2020-10-09,2020-10-02,21,01_moredouble,3.078534
1704,LTU,3.517241,29,2020-10-09,2020-10-02,10,01_moredouble,2.843137
1080,GRC,13.870968,31,2020-10-09,2020-10-02,37,01_moredouble,2.667442


In [31]:
# Worst top 20 countries by absolute deaths
this_week_trend = this_week_trend.sort_values(by = ['deaths_this_week'], ascending=False)
this_week_trend.head(20)

,country_ISO,average,weeks,which_friday,seven_days_ago,deaths_this_week,trend_class,trend
2328,RUS,787.714286,28,2020-10-09,2020-10-02,1165,02_more,1.478963
2588,ESP,1021.500000,32,2020-10-09,2020-10-02,602,04_less,0.589329
872,FRA,956.500000,34,2020-10-09,2020-10-02,502,04_less,0.524830
2744,UKR,156.333333,30,2020-10-09,2020-10-02,497,01_moredouble,3.179104
2796,GBR,1373.935484,31,2020-10-09,2020-10-02,390,05_lesshalf,0.283856
2276,ROU,180.931034,29,2020-10-09,2020-10-02,385,01_moredouble,2.127883
2172,POL,95.566667,30,2020-10-09,2020-10-02,324,01_moredouble,3.390303
612,CZE,29.965517,29,2020-10-09,2020-10-02,191,01_moredouble,6.373993
1444,ITA,1093.424242,33,2020-10-09,2020-10-02,165,05_lesshalf,0.150902
1236,HUN,30.433333,30,2020-10-09,2020-10-02,115,01_moredouble,3.778751
